<a href="https://colab.research.google.com/github/rczhen/faiss_playground/blob/main/index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.pinecone.io/learn/series/faiss/vector-indexes/

This article will explore the pros and cons of some of the most important indexes — Flat, LSH, HNSW, and IVF. We will learn how we decide which to use and the impact of parameters in each index.

We store our vectors in Faiss and query our new Faiss index using a ‘query’ vector. This query vector is compared to other index vectors to find the nearest matches — typically with Euclidean (L2) or inner-product (IP) metrics.

Things to consider:
- Accuracy
- Speed
- Index memory

Indexes
- Flat and accurate
- Locality sensitive hashing (LSH)
- Hierarchical navigable small worlds (HNSW)
- Inverted file index (traditional search)